In [29]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from dataset import Dataset
from utils import Utilities as utils
# for Notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Dataset Creation
Import data into a pandas dataframe using the created class Dataset

In [30]:
ds = Dataset()
files = os.listdir(ds.base_path+"/data/")
ds.createDataset(files)

/home/wiz/Git/LFN-Project/dataset.py:34: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  mob_df = pd.read_csv(mobility_csv, sep=";")
/home/wiz/Git/LFN-Project/dataset.py:34: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  mob_df = pd.read_csv(mobility_csv, sep=";")
/home/wiz/Git/LFN-Project/dataset.py:34: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  mob_df = pd.read_csv(mobility_csv, sep=";")
/home/wiz/Git/LFN-Project/dataset.py:34: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  mob_df = pd.read_csv(mobility_csv, sep=";")


Now we apply the cleanup pre-processing onto the dataframe, in which we also filter the columns we need

In [31]:
# Select only needed for the analysis 
ds.applyPreprocessing(columns=["Participants","Mobility Duration", "Sending Country Code", "Receiving Country Code",
                               "Special Needs", "Fewer Opportunities", "Participant Profile",
                               "Sending Organization", "Receiving Organization", "Receiving Organisation Erasmus Code",
                               "GroupLeader", "Sending Organisation Erasmus Code", "Education Level"])

# Remove all NaN values
ds.cleanDataframe()

Removed 13 columns
Removed 126 rows


Filter Data (If we want)

In [32]:
# # Select only the eramus students researchers from Italy
# ds.applyFilter(column="Sending Country Code", value="IT")
# # Select only students currently under bachelor degree
# ds.applyFilter(column="Education Level", value="ISCED-7", criterion='contains')

# Graph creation
We can now go ahead and create our graph, again using the Dataset class 

In [33]:
from graph import CustomGraph as Graph

The graph are created started from the edges: the networkX library automatically creates the nodes from the edges.

In [34]:
import networkx as nx
import pandas as pd

edges = pd.DataFrame({"source" : ds.df["Sending Country Code"],
                      "target" : ds.df["Receiving Country Code"],
                      "weight" : ds.df["Participants"]
                         })
CountryGraph = nx.from_pandas_edgelist(edges, "source", "target", "weight", nx.DiGraph())


In [35]:
import networkx as nx
import pandas as pd
edges = pd.DataFrame({"source" : ds.df["Sending Organization"],
                      "target" : ds.df["Receiving Organization"],
                      "weight" : ds.df["Participants"]
                         })
UniNodes = nx.from_pandas_edgelist(edges, "source", "target", "weight", nx.DiGraph())

In [36]:
# example of the result with UNIVERSITA DI PADOVA
print("Sum of the exiting participants: ", sum(e[2]['weight'] for e in UniNodes.out_edges("UNIVERSITA DEGLI STUDI DI PADOVA", data=True)))
print("Total Neighbors:",UniNodes.degree(("UNIVERSITA DEGLI STUDI DI PADOVA")))
print("Out Degree:", UniNodes.out_degree("UNIVERSITA DEGLI STUDI DI PADOVA"))
print("In Degree:", UniNodes.in_degree("UNIVERSITA DEGLI STUDI DI PADOVA"))

Sum of the exiting participants:  1419.0
Total Neighbors: 2026
Out Degree: 1364
In Degree: 662


# PageRank
implementation from Networkx library

In [43]:
ranks = sorted(nx.pagerank(UniNodes, weight='weight').items(), key=lambda x: x[1], reverse=True)
UniNodes.pageranks = dict(ranks)

# Standardize the results
mean = np.mean(list(UniNodes.pageranks.values()))
print(f"mean: {mean}")
std = np.std(list(UniNodes.pageranks.values()))
print(f"std: {std}")
UniNodes.pageranks = {k: (v) / std for k, v in UniNodes.pageranks.items()}

mean: 3.1390669437399678e-06
std: 5.439738584221366e-06


In [44]:
rank_df = pd.DataFrame(UniNodes.pageranks, index=['PageRank']).T
rank_df.index.name = 'University'
rank_df.columns = ['PageRank']
display(rank_df)

,PageRank
University,
EUROPASS SRL,197.694898
LFEE EUROPE,172.600642
ENGLISH MATTERS S.L.,143.544239
ITC International TEFL Certificate s.r.o.,139.901118
Executive Training Institute Ltd.,119.104830
...,...
TheDojo.Org.UK LTD,0.486382
INTERCULTURAL EXCHANGE,0.486382
Roma ACCESS Newcastle upon Tyne Association C.I.C.,0.486382


In [45]:
ranks = sorted(nx.pagerank(CountryGraph, weight='weight').items(), key=lambda x: x[1], reverse=True)
CountryGraph.pageranks = dict(ranks)

# Standardize the results
mean = np.mean(list(CountryGraph.pageranks.values()))
print(f"mean: {mean}")
std = np.std(list(CountryGraph.pageranks.values()))
print(f"std: {std}")
CountryGraph.pageranks = {k: (v) / std for k, v in CountryGraph.pageranks.items()}


mean: 0.006024096385542169
std: 0.0060459336100556664


In [60]:
rank_df = pd.DataFrame(CountryGraph.pageranks, index=['Country']).T
rank_df.index.name = 'Country'
rank_df.columns = ['PageRank']
display(rank_df)

,PageRank
Country,
ES,4.940570
DE,3.996802
PL,3.744112
PT,3.649032
RO,3.619386
...,...
GY,0.153594
GQ,0.153594
WS,0.153594


# Clossness Centrality

In [62]:
_cc = sorted(nx.closeness_centrality(CountryGraph, distance='weight').items(), key=lambda x: x[1], reverse=True)
CountryGraph.centrality = dict(_cc)

# Standardize the results
mean = np.mean(list(CountryGraph.centrality.values()))
std = np.std(list(CountryGraph.centrality.values()))
CountryGraph.centrality = {k: (v) / std for k, v in CountryGraph.centrality.items()}

# Visualize data
rank_df = pd.DataFrame(CountryGraph.pageranks, index=['Country']).T
rank_df.index.name = 'Country'
rank_df.columns = ['Centrality Score']
display(rank_df)

,Centrality Score
Country,
ES,4.940570
DE,3.996802
PL,3.744112
PT,3.649032
RO,3.619386
...,...
GY,0.153594
GQ,0.153594
WS,0.153594
